# Random Walker Experiments

Author: K. Voudouris, 2023 (c) All Rights Reserved.

Contact: kv301@cam.ac.uk; k.voudouris14@googlemail.com; [Twitter @KozzyVoudouris](https://twitter.com/KozzyVoudouris); [GitHub @kozzy97](https://github.com/kozzy97)

Date: July 2023

This script runs a series of random walkers on the object permanence tests and stores the results in a MySQL database. It relies on a few things:

1. All the dependencies are installed, particularly that animalai is installed properly. I recommend using a conda environment and setting up an ipykernel for running this notebook.
2. AnimalAI is installed as an executable in the `env` folder.
3. A recent installation of MySQL, configured with a database, user, and password, as well as the local (or remote) address to store on. MySQL WorkBench is a good IDE for interacting with MySQL (this was created with WorkBench 8.0)
4. A CSV file in the same directory as this notebook called `databaseConnectionDetails.csv`, containing columns `database_name`, `hostname`, `username`, and `password` for database connection, with the values in the next row. This is gitignored.

In [16]:
import keyboard
import random
import os
import time

from animalai.envs.environment import AnimalAIEnvironment
from animalai.envs.actions import AAIActions, AAIAction
from gym_unity.envs import UnityToGymWrapper

import sys
sys.path.append('../src')

from randomWalkers import RandomWalker #import the random walker class
from yamlHandling import find_yaml_files #this function finds the yaml files in a directory.
from yamlHandling import yaml_combinor #this function combines a batch of yaml files and saves the output in a temporary folder. This means we can run inference on batches of tests at once.
from mysqlConnection import databaseConnector #this function permits connection to a mysql database using a CSV file containing details of the db connection.
from mysqlConnection import agentToDB #this function takes a dictionary and ingresses it into a table
from mysqlConnection import removePreviouslyRunInstances #this function takes a set of yaml files and task names and removes any that have already got results in the database.
from mysqlConnection import selectID #this function finds the integer ID for a table given a particular column name and value

## Database Connection

A function for connecting to the database.

In [ ]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

mycursor.close()

print("Connection checked and closed.")

## Paths

Provide the paths to the directory containing the configs being tested over, as well as the path the animal ai environment. Finally, provide a location for generating temporary files of combined configs. This defaults to the parent directory of the github repository, to prevent results being pushed accidentally.

In [ ]:
configuration_folder = "../../configs/tests_agents"

env_path = "../../env/AnimalAI"

temp_folder_location = "../../.."

## Add All Tasks In Directory To Database

Iterate through the directory and find yaml files and their task names.

In [ ]:
rerunInstanceTable = False

mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

yaml_files, task_names = find_yaml_files(configuration_folder)

if rerunInstanceTable:
    dropTable = "DROP TABLE IF EXISTS randomwalkerinstanceresults, randomwalkerintrainstanceresults, instances;"
    mycursor.execute(dropTable)
    
    sql = "CREATE TABLE instances(instanceid INT AUTO_INCREMENT PRIMARY KEY, instancename VARCHAR(750) UNIQUE NOT NULL);"
    mycursor.execute(sql)

for instance in task_names:
    try:
        insertQuery = "INSERT INTO instances(instancename) VALUES('" + str(instance) + "');"
        mycursor.execute(insertQuery)
        connection.commit()
    except:
        print(f"Task {instance} has already been added to this table. Moving to next.")

mycursor.close()



## Agents

Dictionaries of parameters to define some random walker agents. Note that these agents move quite slowly. They take one step forwards then pause for one step. This is to reduce the effect of momentum in the environment so their paths are more accurate.

The first agent is a simple fixed-step random walker. The walker moves 15 steps forwards, and then turns for 10 steps, with a probability of 0.5 of turning left or right. This serves as a very simple baseline.

In [ ]:
fixed_action_walker = {'saccade_distribution' : 'fixed',
                       'angle_distribution' : 'fixed',
                       'max_saccade_length' : 15,
                       'max_angle_steps': 10,
                       'angle_fixed_randomise_turn': True,
                       'aai_seed' : 2023,
                       'agent_tag' : 'Random Walker Fixed Forwards Saccade 15 Angle 10'}

The next agent is a common model of foraging in animals. It is a Rayleight flight. Saccades are sampled from the normal distribution with a mean of 15 and a standard deviation of 5. If negative values are sampled the agent goes backwards that many steps, so the agent will tend to go forwards but occasionally go backwards. Angles are sampled uniformly between 1 and 60 steps of turning. Each turn is 6 degrees, making 60 steps be a full turn. Choice of left and right is randomised with probability 0.5.

In [ ]:
rayleigh_flight = {'saccade_distribution' : 'normal',
                   'angle_distribution' : 'uniform',
                   'saccade_norm_mu' : 15,
                   'saccade_norm_sig' : 5,
                   'max_angle_steps': 60,
                   'angle_fixed_randomise_turn': True,
                   'backwards_action' : True,
                   'aai_seed' : 2023,
                   'agent_tag' : 'Random Walker Rayleigh Flight Norm Saccade mu 15 sig 5 Uniform Angles 1 to 60'}

The next agent is another common model of foraging in animals. It is a levy/cauchy flight. Saccades are sampled from the cauchy distribution, centred around 15. If negative values are sampled the agent goes backwards that many steps, so the agent will tend to go forwards but occasionally it will go backwards. Angles are sampled uniformly between 1 and 60 steps of turing. Each turn is 6 degrees, making 60 steps be a full turn. Choice of left and right is randomised with probability 0.5.

In [ ]:
levy_flight = {'saccade_distribution' : 'cauchy',
               'angle_distribution' : 'uniform',
               'saccade_cauchy_mode' : 15,
               'max_angle_steps': 60,
               'angle_fixed_randomise_turn': True,
               'backwards_action' : True,
               'aai_seed' : 2023,
               'agent_tag' : 'Random Walker Levy Flight Cauchy Saccade mode 15 Uniform Angles 1 to 60'}

In [ ]:
rerunAgentTable = False

mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

if rerunAgentTable:
    dropTable = "DROP TABLE IF EXISTS randomwalkerinstanceresults, randomwalkerintrainstanceresults, randomwalkers;"
    mycursor.execute(dropTable)
    
    sql = "CREATE TABLE `randomwalkers` (`agentid` INT AUTO_INCREMENT PRIMARY KEY, `agent_tag` VARCHAR(300), `aai_seed` INT, `max_saccade_length` INT,  `max_angle_steps` INT,  `saccade_distribution` VARCHAR(10), `angle_distribution` VARCHAR(10), `saccade_norm_mu` FLOAT(8), `saccade_norm_sig` FLOAT(8), `saccade_beta_alpha` FLOAT(8), `saccade_beta_beta` FLOAT(8), `saccade_cauchy_mode` FLOAT(8), `saccade_gamma_kappa` FLOAT(8), `saccade_gamma_theta` FLOAT(8), `saccade_weibull_alpha` FLOAT(8), `saccade_poisson_lambda` FLOAT(8), `angle_fixed_randomise_turn` BOOL, `angle_norm_mu` FLOAT(8), `angle_norm_sig` FLOAT(8), `angle_beta_alpha` FLOAT(8), `angle_beta_beta` FLOAT(8), `angle_cauchy_mode` FLOAT(8), `angle_gamma_kappa` FLOAT(8), `angle_gamma_theta` FLOAT(8), `angle_weibull_alpha` FLOAT(8), `angle_poisson_lambda` FLOAT(8), `angle_correlation` FLOAT(8), `backwards_action` BOOL, UNIQUE(agent_tag, aai_seed));"
    mycursor.execute(sql)

mycursor.close()

In [ ]:
agent_dict_list = [fixed_action_walker, rayleigh_flight, levy_flight]

seeds_to_run = [2023, 1997, 356, 1815, 3761, 1184, 1956, 1804, 2050, 1967] #10 seeds corresponding to eventful years.

In [ ]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

for agent in agent_dict_list:
    for seed in seeds_to_run:
        agent['aai_seed'] = seed
        agentToDB(mycursor, agent, table_name = "randomwalkers")

connection.commit()

mycursor.close()

## Run Inference And Store

Need to iterate through the dictionaries and run inference.

In [ ]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

rebuildInstanceResultsTables = False

if rebuildInstanceResultsTables:
    print("Rebuilding results tables, dropping if they already exist.")

    dropInstanceResultsTables = "DROP TABLE IF EXISTS randomwalkerinstanceresults, randomwalkerintrainstanceresults;"
    mycursor.execute(dropInstanceResultsTables)
    
    createInstanceTable = "CREATE TABLE randomwalkerinstanceresults(instanceid INT NOT NULL, agentid INT NOT NULL, finalreward FLOAT(53), FOREIGN KEY (instanceid) REFERENCES instances(instanceid), FOREIGN KEY(agentid) REFERENCES randomwalkers(agentid), PRIMARY KEY (instanceid, agentid));"
    mycursor.execute(createInstanceTable)

    createIntraInstanceTable = "CREATE TABLE randomwalkerintrainstanceresults(instanceid INT NOT NULL, agentid INT NOT NULL, step INT NOT NULL, actiontaken INT NOT NULL, stepreward FLOAT(53), xvelocity FLOAT(32), yvelocity FLOAT(32), zvelocity FLOAT(32), xpos FLOAT(32), ypos FLOAT(32), zpos FLOAT(32), FOREIGN KEY (instanceid) REFERENCES instances(instanceid), FOREIGN KEY(agentid) REFERENCES randomwalkers(agentid), PRIMARY KEY(instanceid, agentid, step));"
    mycursor.execute(createIntraInstanceTable)

    print("Tables: randomwalkerinstanceresults and randomwalkerintrainstanceresults have been successfully built.")

mycursor.close()


Define a function to run the experiments. This takes an agent dictionary and first checks whether any results have been recorded for it. If not, then it proceeds with testing. It does testing in batches, generating a temporary yml file to run training on and storing the final episode reward, as well as the intra-instance results.

In [ ]:
def runRandomWalkerAndStore(cur, con, batch_size: int, agent_dict: dict, yaml_files, task_names, temp_folder_location, agent_inference = False, port_base = 6600, randomise_port = True):
    
    # first, check if this agent has been added to the DB already

    agentid = selectID(cur, id_name = "agentid", table_name = "randomwalkers", WHERE_column = "agent_tag", WHERE_clause = agent_dict['agent_tag'], secondary_WHERE_column = "aai_seed", secondary_WHERE_clause = agent_dict['aai_seed'])

    try:
        task_names, yaml_files = removePreviouslyRunInstances(cur = cur, yaml_files=yaml_files, task_names=task_names, agentid=agentid, agent_table = "randomwalkers", agent_instance_results_table = "randomwalkerinstanceresults")
    except:
        print("Running on all files.")

    # now proceed with testing
    yaml_index = 0

    if randomise_port:

        port = port_base + yaml_index + random.randint( #create random base port.
            0, 9000
            )
        
    else:
        port = port_base + yaml_index
        
    batch_counter = 0

    #set seed
    random.seed(agent_dict['aai_seed'])

    if len(yaml_files) > 0:
        for yaml_index in range(0, len(yaml_files), batch_size):

            if (yaml_index + batch_size) > len(yaml_files) or batch_size > len(yaml_files):
                upper_bound = len(yaml_files)
            else:
                upper_bound = (yaml_index + batch_size)

            print(f"Running inferences on batch {batch_counter + 1} of {batch_size} files of total {len(yaml_files)}. {len(yaml_files) - (batch_size * (batch_counter + 1))} instances to go.")

            batch_files = yaml_files[yaml_index:upper_bound]

            batch_file_names = task_names[yaml_index:upper_bound]

            batch_temp_file_name = f"TempConfig_{agent_dict['agent_tag']}_{agent_dict['aai_seed']}_{yaml_index}.yml"

            config_file_path = yaml_combinor(file_list = batch_files, temp_file_location=temp_folder_location, stored_file_name = batch_temp_file_name)

            print("Opening AAI Environment.")

            temp_port = port + yaml_index # increment through ports to prevent calling the same socket.

            aai_env = AnimalAIEnvironment( 
                inference=agent_inference, #Set true when watching the agent
                seed = agent_dict['aai_seed'],
                worker_id=temp_port,
                file_name=env_path,
                arenas_configurations=config_file_path,
                base_port=temp_port,
                useCamera=False,
                resolution=4, #make resolution small to improve processing speed - random walkers don't need anything.
                useRayCasts=False,
                no_graphics=True
            )

            env = UnityToGymWrapper(aai_env, uint8_visual=False, allow_multiple_obs=True, flatten_branched=True)

            obs = env.reset()  

            agent = RandomWalker() # initialise agent class

            for key, value in agent_dict.items(): #set the agent attributes to be whatever is in the dictionary, and default otherwsise.
                if hasattr(agent, key):
                    setattr(agent, key, value)

            for _instance in range(len(batch_files)): 

                #get instance ID
                instanceid = selectID(cur, id_name = "instanceid", table_name = "instances", WHERE_column = "instancename", WHERE_clause = batch_file_names[_instance])

                #prepare to run instance
                done = False

                episodeReward = 0

                step_counter = 0
    
                while not done:

                    saccade_list = agent.get_num_steps_saccade()
            
                    for action in saccade_list:
            
                        obs, reward, done, info = env.step(int(action))

                        env.render()
             
                        step_counter += 1

                        episodeReward += reward

                        try:
                            intraInstanceQuery = intraInstanceQuery = f"INSERT INTO randomwalkerintrainstanceresults(instanceid, agentid, step, actiontaken, stepreward, xvelocity, yvelocity, zvelocity, xpos, ypos, zpos) VALUES ({instanceid}, {agentid}, {step_counter}, {action}, {float(episodeReward)}, {obs[0][1]}, {obs[0][2]}, {obs[0][3]}, {obs[0][4]}, {obs[0][5]}, {obs[0][6]});"
                            cur.execute(intraInstanceQuery)
                            con.commit()
         
                        except:
                            print(f"There's something wrong with this step. Here's the query {intraInstanceQuery}")
                            pass

                        if done:
                            obs=env.reset()
                            print(f"Episode Reward: {episodeReward}")
                            done = True #to be sure.
                            break #break the for loop early
                    
                    
                    if not done: # only do turns if episode not done yet.

                        if 'num_angle_steps' not in locals() and agent.angle_distribution == 'normal': #if no turns have been done yet and using normal distribution, then set the central moment to be the prespecified normal_mu
                            prev_angle_central_moment = agent.angle_norm_mu
                        elif 'num_angle_steps' not in locals() and agent.angle_distribution == 'cauchy': #as above
                            prev_angle_central_moment = agent.angle_cauchy_mode
                        elif 'num_angle_steps' in locals():
                            prev_angle_central_moment = num_angle_steps #if turns have been done, then the central moment is whatever number of steps was provided before.
                        else:
                            prev_angle_central_moment = 0

                        angle_list, num_angle_steps = agent.get_num_steps_turn(prev_angle_central_moment)
        
                        for action in angle_list:
         
                            obs, reward, done, info = env.step(int(action))
            
                            step_counter += 1

                            env.render()

                            episodeReward += reward

                            try:
                                intraInstanceQuery = f"INSERT INTO randomwalkerintrainstanceresults(instanceid, agentid, step, actiontaken, stepreward, xvelocity, yvelocity, zvelocity, xpos, ypos, zpos) VALUES ({instanceid}, {agentid}, {step_counter}, {action}, {float(episodeReward)}, {obs[0][1]}, {obs[0][2]}, {obs[0][3]}, {obs[0][4]}, {obs[0][5]}, {obs[0][6]});"
                                cur.execute(intraInstanceQuery)
                                con.commit()
         
                            except:
                                print(f"There's something wrong with this step. Here's the query {intraInstanceQuery}")
                                pass
                        

                            if done:
                                print(F"Episode Reward: {episodeReward}")
                                obs=env.reset()
                                done = True #to be sure.
                                break

                try:
                    insertInstanceResults = f"INSERT INTO randomwalkerinstanceresults(instanceid, agentid, finalreward) VALUES ({instanceid}, {agentid}, {episodeReward});"
                    cur.execute(insertInstanceResults)
                    con.commit()
                    print("Pushing results to database.")
                except:
                    print("It looks like this agent has already been tested on this instance.")

                    
            env.close()

            batch_counter += 1

            os.remove(config_file_path)

            random.seed() #set a random new seed for port allocation.
            
            print("Moving to next batch.")

    else:
        print("This agent has already been run and is in the database. Skipping so as not to waste time. If you suspect that the agent has not been fully evaluated on all tests, you may want to restart the instances for that agent.")
     

In [ ]:
def run_agent_on_instance_wrapper(seed, agent, yaml_batch_size=1, port_base = 6600, randomise_port = True):
    agent['aai_seed'] = seed

    print(f"Running {agent['agent_tag']} on seed {seed}.")
    
    mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

    runRandomWalkerAndStore(mycursor, connection, yaml_batch_size, agent_dict=agent, yaml_files=yaml_files, task_names=task_names, temp_folder_location=temp_folder_location, agent_inference=False, port_base = port_base, randomise_port = randomise_port)

    mycursor.close()

In [ ]:
yaml_batch_size = 1 #problem with task ordering, so having to do batches of 1. Much slower...
counter = 0
inf_loop = True

while inf_loop:
    try:
        for seed in seeds_to_run:
            for agent_dictionary in agent_dict_list:
                if keyboard.is_pressed('q'):
                    print(f"Loop stopped by pressing 'q'.")
                    inf_loop = False
                    break
                counter += 1
                adhoc_port = counter*4500
                run_agent_on_instance_wrapper(seed, agent_dictionary, yaml_batch_size=yaml_batch_size, port_base = adhoc_port, randomise_port = False)
                print("Moving to next seed.")
            
            if not inf_loop:
                break
            print("Moving to next agent.")
    except:
        print("Sockets were occupied. Waiting 10 seconds and starting again.")
        if keyboard.is_pressed('q'):
                    print(f"Loop stopped by pressing 'q'.")
                    inf_loop = False
                    break
        time.sleep(10)